# IBM Data Science Specialization Course 9

This notebook will be mainly used for the capstone project.

In [1]:
import pandas as pd
import numpy as np

print('Hello Capstone Project Course!')

Hello Capstone Project Course!


# Course 9 week 3 assignment: Toronto

Scrape the Wikipedia table at https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M (retrieved 24 Feb 2019) into a pandas DataFrame.

## First try: parse Wikipedia using py-wikimarkup

Initial how-to instructions from https://stackoverflow.com/questions/15724034/how-to-convert-wikipedia-wikitable-to-python-pandas-dataframe (retrieved 24 Feb 2019).

In [2]:
!pip install py-wikimarkup
!pip install pyquery

  Running setup.py bdist_wheel for py-wikimarkup ... done
  Stored in directory: /home/dsxuser/.cache/pip/wheels/50/a3/eb/47f4014d6bd8d2d233b552150d8ba308299cdf0bd2046ddd91
Successfully built py-wikimarkup
Requirement not upgraded as not directly required: lxml>=2.1 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from pyquery)


In [3]:
import wikimarkup
import pandas as pd
from pyquery import PyQuery

def get_tables(wiki):
    html = PyQuery(wikimarkup.parse(wiki))
    frames = []
    for table in html('table'):
        data = [[x.text.strip() for x in row]
                for row in table.getchildren()]
        df = pd.DataFrame(data[1:], columns=data[0])
        frames.append(df)
    return frames


ImportError: cannot import name 'parse'

Unfortunately, a pre-installed "parse" package in IBM Watson Studio conflicts with the "parse.py" file name inside wikimarkup.

## Second try: parse Wikipedia using BeautifulSoup

Following https://medium.com/analytics-vidhya/web-scraping-wiki-tables-using-beautifulsoup-and-python-6b9ea26d8722 (retrieved 24 Feb 2019)

Gettin HTML of the Wikipedia page:

In [12]:
import requests
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
website_url = requests.get(url).text

from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url, 'lxml')
print("{}\n\n   [...]\n\n{}".format(soup.prettify()[:500], soup.prettify()[-500:]))

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_po

   [...]

:"Contributors to Wikimedia projects"},"publisher":{"@type":"Organization","name":"Wikimedia Foundation, Inc.","logo":{"@type":"ImageObject","url":"https:\/\/www.wikimedia.org\/static\/images\/wmf-hor-googpub.png"}},"datePublished":"2004-03-20T10:02:13Z","dateModified":"2019-01-04T18:32:45Z","headline":"Wikimedia list article"}
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgBackendResponseTime":113,"wgHostname":"mw1329"});});
  </script>
 </bo

This library seems to function OK.

The needed data is contained in the first (and only) table, and is rendered with HTML style "wikitable sortable". Parse it:

In [14]:
My_table = soup.find('table', { 'class' : 'wikitable sortable' })
print("{}\n\n   [...]\n\n{}".format(str(My_table)[:500], str(My_table)[-500:]))

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td>

   [...]

/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/The_Queensway" title="The Queensway">The Queensway West</a>
</td></tr>
<tr>
<td>M8Z</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td>Royal York South West
</td></tr>
<tr>
<td>M8Z</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Bloor" title="Bloor">South of Bloor</a>
</td></tr>
<tr>
<td>M9Z</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
</tb

## Clean data for postal code, borough, and neighborhood

Now loop through all rows from that table. For each row:
1. Ignore if it is the header row (no "TD" tags),
2. strip leading and trailing whitespace (including line breaks),
3. Ignore if the borough is "Not assigned",
4. If the borough is given but neighborhoos is "Not assigned", fill in the borough as neighborhood name.
Finally, build a new pandas DataFrame from the list items collected.

In [25]:
l = []
unassigned_pc = []

table_rows = My_table.find_all('tr')
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td]
    if len(row) < 1:
        print("ignoring empty row")
    elif row[1] == 'Not assigned':
        unassigned_pc.append(row[0])
    else:
        if row[2] == 'Not assigned':
            row[2] = row[1]
            print('assigning neighborhood same as borough: {}'.format(row[2]))
        l.append(row)

print("ignoring unassigned postal codes:{}".format(unassigned_pc))
        
My_df = pd.DataFrame(l, columns=["PostalCode", "Borough", "Neighborhood"])
My_df.head()

ignoring empty row
assigning neighborhood same as borough: Queen's Park
ignoring unassigned postal codes:['M1A', 'M2A', 'M8A', 'M2B', 'M7B', 'M8B', 'M2C', 'M7C', 'M8C', 'M2E', 'M3E', 'M7E', 'M8E', 'M9E', 'M2G', 'M3G', 'M7G', 'M8G', 'M9G', 'M7H', 'M8H', 'M9H', 'M7J', 'M8J', 'M9J', 'M7K', 'M8K', 'M9K', 'M7L', 'M8L', 'M7M', 'M8M', 'M7N', 'M8N', 'M3P', 'M7P', 'M8P', 'M3R', 'M8R', 'M2S', 'M3S', 'M7S', 'M8S', 'M9S', 'M2T', 'M3T', 'M6T', 'M7T', 'M8T', 'M9T', 'M2V', 'M3V', 'M6V', 'M7V', 'M2W', 'M3W', 'M6W', 'M7W', 'M2X', 'M3X', 'M6X', 'M7X', 'M9X', 'M1Y', 'M2Y', 'M3Y', 'M5Y', 'M6Y', 'M9Y', 'M1Z', 'M2Z', 'M3Z', 'M4Z', 'M5Z', 'M6Z', 'M7Z', 'M9Z']


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


## Aggregate neighborhoods by borough and postal code

Concatenate neighborhoods grouped by postal code and borough.

df_grouped = My_df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(lambda x: "%s" % ', '.join(x)).reset_index()
df_grouped.head()

## Result: Number of rows in Toronto neighborhood DataFrame

In [45]:
print("Numver of records in cleaned, aggregated Toronto neighborhoods scraped from Wikipedia:")
print("df.shape={},\nnumber of rows={}".format(df_grouped.shape, df_grouped.shape[0]))

Numver of records in cleaned, aggregated Toronto neighborhoods scraped from Wikipedia:
df.shape=(103, 3),
number of rows=103


In [46]:
!pip install geocoder

Requirement not upgraded as not directly required: geocoder in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: future in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: click in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: ratelim in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->geocoder)
Requirement not upgraded as not directly required: i

In [47]:
import geocoder # import geocoder

# testing geocoder
lat_lng_coords = None
cnt = 0

# loop until you get the coordinates
while(lat_lng_coords is None and cnt < 20):
    print('attempt {}'.format(cnt))
    cnt += 1
    g = geocoder.google('Toronto, Canada')
    lat_lng_coords = g.latlng

if lat_lng_coords is None:
    print('API calls unsuccessful')
else:
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    print("{} / {}".format(latitude, longitude))


attempt 0
attempt 1
attempt 2
attempt 3
attempt 4
attempt 5
attempt 6
attempt 7
attempt 8
attempt 9
attempt 10
attempt 11
attempt 12
attempt 13
attempt 14
attempt 15
attempt 16
attempt 17
attempt 18
attempt 19
API calls unsuccessful


In [48]:
## TODO continue here